# 第 1 章：线性代数（Linear Algebra for Runtime Inference）

## 1.1 线性代数在推理加速中的角色

在深度学习推理中，绝大部分“重计算”都可以抽象为：

- 向量内积（dot product）
- 矩阵–向量乘法（GEMV）
- 矩阵–矩阵乘法（GEMM）
- 卷积经变换后变成的 GEMM（im2col / Winograd）

例如：

- 全连接层（Fully Connected）：  
  $y = W x + b$
- Transformer MLP：  
  $Y = \sigma(X W_1 + b_1) W_2 + b_2$
- 自注意力（Self-Attention）：  
  $Q = X W_Q, \quad K = X W_K, \quad V = X W_V$  
  $A = \text{softmax}(QK^\top / \sqrt{d_k})$  
  $O = A V$

**【工程视角】**  
推理加速库（cuBLAS、CUTLASS、MKL、QNNPACK、TensorRT 等）的核心任务，就是：

> 让这些矩阵/向量运算在特定硬件上尽可能高效地执行。

理解线性代数 → 理解：

- 为什么所有人都在说 GEMM
- 为什么 low-rank / SVD / LoRA 能够“白嫖”加速
- 为什么 Conv2d 经常被变成 GEMM
- 为什么 Attention 的优化集中在 QKᵀ / AV 上

---

## 1.2 从内积到 GEMM：推理的基本算子

### 1.2.1 内积（dot product）

给定两个向量 $a,b \in \mathbb{R}^d$，它们的内积为：

$$
\langle a, b \rangle = \sum_{i=1}^d a_i b_i
$$


这是最小的“乘–加”单元。

**【硬件视角】**  
现代 CPU/GPU/NPU 都有 **FMA (Fused Multiply-Add)** 指令：

$$
\text{FMA}(a,b,c) = a \times b + c
$$


dot product 可以写成：

```text
acc = 0
for i in range(d):
    acc = fma(a[i], b[i], acc)
```

- FMA 是矩阵乘法的底层原语（primitive）
- 越能让硬件长时间“刷 FMA”，推理吞吐越高
- 数学上的意义：
$\text{round}\big(\text{round}(a\times b) + c \big)
\neq
\text{round}(a\times b + c)$
FMA 使用右边的（更精确）。

---

### 1.2.2 矩阵乘法（GEMM）

GEMM 的一般形式：

$$
C = \alpha A B + \beta C
$$


其中：

- $A \in \mathbb{R}^{M \times K}$
- $B \in \mathbb{R}^{K \times N}$
- $C \in \mathbb{R}^{M \times N}$
- $\alpha, \beta$ 为标量（常见情况：$\alpha=1, \beta=0$）

展开单个元素：

$$
c_{ij} = \alpha \sum_{k=1}^K a_{ik} b_{kj} + \beta c_{ij}^{\text{(old)}}
$$


这就是：**多次 FMA 的二维版本**。

**【工程视角】**  
- 全连接层 = 多个 GEMM
- Attention = 2 个大 GEMM（QKᵀ 和 AV）+ 若干小算子
- Conv2d 经过变换后 = GEMM
- 推理加速的大头，就是把不同算子统一映射到高效 GEMM 内核上


小结
| 网络组件            | 数学形式     | 最终变成          |
| --------------- | -------- | ------------- |
| 全连接层 FC         | Wx + b   | GEMM          |
| 卷积 conv2d       | K * X    | im2col → GEMM |
| Attention QKᵀ   | QKᵀ      | GEMM          |
| Attention AV    | AV       | GEMM          |
| Transformer MLP | XW₁, XW₂ | GEMM          |
| Embedding       | 查表       | 选行（类似 GEMM）   

附： 为什么说“Embedding 类似 GEMM”？

虽然 数学上 Embedding 是选行，
但它可以看成一个非常稀疏的矩阵乘法。
给一个 embedding matrix（词向量矩阵）
$E \in \mathbb{R}^{V \times D}$
- V = vocabulary size（如 50k）

- D = hidden dimension（如 1024）

每一行 $𝐸_i$ 就是一个 token 的向量。当一个 token 的 ID = i 时，embedding 做的事情是：$\text{Embedding}(i) = E_{i}$
没有乘法，没有矩阵运算，所以 Embedding 的 FLOPs ~ 0（几乎 0）。
真正的成本是：

- 内存访问（memory bandwidth）

- 随机访问（random access）

- cache miss（embedding layer 很容易 miss cache）

假设 token id = 3， 这时可以构造一个 size V 的 one-hot 向量：$x = [0,0,0,1,0,0,\dots]^T \in \mathbb{R}^V$

Embedding 的输出就是：$y = x^T E \in \mathbb{R}^D$

展开来看：$y = \sum_{i=1}^V x_i E_i$

但因为只有一个 $X_i = 1$，其余全是 0： $y = E_3$

这本质上是一种特殊的 GEMM：$Y = X E$
其中 X 是 batch 个 one-hot 向量。

只是：

X 中绝大部分值是 0， X 只有一个 1， 所以矩阵乘法退化成“选行”.

这么理解的好处：
Embedding 从数学上可以视为 GEMM 的特例（稀疏 GEMM）

这让我们可以：

- 使用矩阵观点分析它

- 使用 GEMM 的 layout、tiling 理解其优化

- 把 embedding 也当成一个“矩阵算子”，可以融合、量化、cache友好化

总结一下，Embedding 的瓶颈不是算力（不是 FMA），而是：

- 内存带宽（bandwidth）

- 随机访问（random access patterns）

- cache line miss

因此：

常见看到优化策略是：

- 把 embedding matrix rearrange（行压缩）

- 把常用 token 提前放在 cache-friendly 区域

- 更小 embedding（量化、减少维度）

- batching lookups


---

### 1.2.3 张量运算：线性代数的多维版本

张量（tensor）本质就是多维数组，常见是 rank-3 / rank-4：

- rank-3：$B \times T \times D$（batch × seq_len × hidden_dim）
- rank-4：$B \times C \times H \times W$（用于卷积）

张量运算（einsum、batched matmul）最终都会被分解成：

- 一堆 reshape / transpose
- 若干矩阵乘法（batched GEMM）
- 再 reshape 回想要的形状

> 所以“学线性代数”不仅是学矩阵，还要学会**如何把高维张量 reshape 成矩阵**以便使用高效 GEMM。


## 1.3 低秩近似（Low-Rank Approximation）与 SVD

### 1.3.1 问题背景

给定一个大的权重矩阵 $W \in \mathbb{R}^{m \times n}$，例如：

- Transformer MLP 的权重（如 4096 × 11008）
- Attention 中的投影矩阵（如 4096 × 4096）

在很多实际模型中，$W$ 的“有效秩（effective rank）”远小于 $\min(m,n)$：
- 也就是说，“信息”集中在少数几个方向上。

这时我们希望用一个秩为 $k \ll \min(m,n)$ 的矩阵 $W_k$ 来近似 $W$：

$$
\min_{\text{rank}(X) \le k} \ \|W - X\|_F
$$


这个问题的解析解由 **奇异值分解（SVD）** 给出。

---

### 1.3.2 奇异值分解（SVD）的形式

$$
W = U \Sigma V^\top
$$


- $U \in \mathbb{R}^{m \times m}$：列向量为左奇异向量
- $V \in \mathbb{R}^{n \times n}$：列向量为右奇异向量
- $\Sigma \in \mathbb{R}^{m \times n}$：对角线为奇异值 $\sigma_1 \ge \sigma_2 \ge \dots \ge \sigma_r > 0$，其余为 0

截断到前 $k$ 个奇异值：

$$
W_k = U_k \Sigma_k V_k^\top
$$


- $U_k \in \mathbb{R}^{m \times k}$
- $\Sigma_k \in \mathbb{R}^{k \times k}$
- $V_k \in \mathbb{R}^{n \times k}$

这是一个秩为 $k$ 的矩阵。

---

### 1.3.3 最优低秩近似定理（Eckart–Young–Mirsky）

**定理：**  
在 Frobenius 范数下，秩不超过 $k$ 的矩阵中离 $W$ 最近的是 $W_k$：

$$
W_k = \arg\min_{\text{rank}(X) \le k} \|W - X\|_F
$$


而误差大小刚好等于被截掉的奇异值的平方和：

$$
\|W - W_k\|_F^2 = \sum_{i=k+1}^r \sigma_i^2
$$


**【直观解释】**  
奇异值越大，对矩阵“能量”的贡献越大；截断后，丢掉的能量恰好是对应奇异值平方的总和。

---

### 1.3.4 用两个小矩阵实现低秩近似

在工程中，为了减少 FLOPs，通常把 $W_k$ 拆成两个更小的矩阵相乘：

$$
W_k = A B, \quad A \in \mathbb{R}^{m \times k}, \ B \in \mathbb{R}^{k \times n}
$$


一种常见构造是：

$$
A = U_k \Sigma_k^{1/2}, \quad B = \Sigma_k^{1/2} V_k^\top
$$


于是：

$$
W x \approx W_k x = A (B x)
$$


- 原始计算：$W x$ 是一个 $m \times n$ 的矩阵–向量乘法，复杂度 $O(mn)$
- 分解后：
  - 先算 $z = B x$（维度 $k$）
  - 再算 $A z$
  - 总复杂度 $O(kn + mk)$

如果 $k \ll \min(m,n)$，总体 FLOPs 大幅减少。

**【工程视角】**  
- 这就是很多 **SVD-based compression / low-rank factorization / LoRA** 内核的数学基础。
- 在模型部署时，你会看到某些层被替换成两层小的 Linear：`Linear(d_in → k)` + `Linear(k → d_out)`。

---

### 1.3.5 文字图示（矩阵结构的可视化）

原始矩阵 $W$：

```text
W (m×n)
+------------------------+
| █ █ █ █ █ █ █ █ █ █ █  |
| █ █ █ █ █ █ █ █ █ █ █  |
| █ █ █ █ █ █ █ █ █ █ █  |
| █ █ █ █ █ █ █ █ █ █ █  |
|         ···            |
+------------------------+
```

低秩分解后：

```text
A (m×k)          B (k×n)
+------+       +------------------+
| █ █ |       | █ █ █ █ █ █ █ █   |
| █ █ |   x   | █ █ █ █ █ █ █ █   |
| █ █ |       |        ···        |
| █ █ |       +------------------+
+------+
```

推理时：

1. 先计算 $z = B x$（维度 k）  
2. 再计算 $y = A z$

当 k 比 m,n 小很多时，这是显著的加速。

---

### 1.3.6 在推理加速中的典型用法

1. **MLP 权重低秩分解**  
   - 原始：4096×11008 的巨矩阵  
   - 分解：4096×k 和 k×11008，k=1024 或更小  
2. **LoRA（Low-Rank Adaptation）**  
   - 把更新量 $\Delta W$ 参数化为 $A B$，rank 通常很小（如 8、16）  
   - 对部署来说，推理时多了两层小矩阵乘法
3. **KV Cache 压缩 / 投影**  
   - 用低秩映射降低 KV 维度，从而减少内存和带宽
4. **结构化剪枝的替代方案**  
   - 相比直接硬剪通道，低秩分解是更平滑的维度削减方式

> 在这类做边缘推理的场景中：  
> - 你会希望在 **不大幅降精度** 的前提下，**显著减少 MLP 和投影层的 FLOPs 和参数量**，低秩分解是最主流的数学工具之一。


## 1.4 矩阵范数与误差度量

在做任何压缩（低秩、剪枝、量化）时，你都需要一个“度量标准”来衡量：

- 压缩前后的权重差异有多大？
- 这会带来多大的输出误差？

矩阵范数提供了这些度量工具。

---

### 1.4.1 Frobenius 范数（整体能量）

$$
\|W\|_F = \sqrt{\sum_{i,j} w_{ij}^2}
$$


对应于把矩阵当成一个长向量后的 $L_2$ 范数。

- 它衡量整体“能量”（energy）
- 在 SVD 中有非常漂亮的性质：

$$
\|W\|_F^2 = \sum_{i=1}^r \sigma_i^2
$$


而对低秩近似：

$$
\|W - W_k\|_F^2 = \sum_{i=k+1}^r \sigma_i^2
$$


**【工程应用】**  
- 可以通过奇异值快速估计压缩误差的上界
- 可以比较不同 $k$ 值下的误差，做 bit–accuracy tradeoff

---

### 1.4.2 谱范数（operator norm / 2-范数）

$$
\|W\|_2 = \sigma_{\max}(W)
$$


- 等于最大的奇异值
- 反映：$W$ 作为线性算子能把向量放大的最大比例

具体来说：

$$
\|W\|_2 = \max_{\|x\|_2 = 1} \|W x\|_2
$$


**【工程意义】**  
- 如果 $\|W\|_2$ 非常大，则对输入的小扰动非常敏感  
**解释**：当矩阵的谱范数（spectral norm），即 $|W|_2$ 非常大时，它意味着这个矩阵作为一个线性变换，能够将输入向量“放大”的最大比例非常大。

谱范数的定义是：

$$
\|W\|_2 = \max_{\|x\|_2 = 1} \|W x\|_2
$$

这意味着，对于任何非零向量 $x$，

$$|W x|_2 \le |W|_2 |x|_2$$

现在，考虑一个输入向量 $x_0$ 受到一个小扰动 $\Delta x$。那么，输出的改变将是 $W(x_0 + \Delta x) - W x_0 = W \Delta x$。

输出扰动的范数是 $|W \Delta x|_2$。根据上面的不等式，我们可以得到：

$$|W \Delta x|_2 \le |W|_2 |\Delta x|_2$$

如果 $|W|_2$ 非常大，即使输入的扰动 $|\Delta x|_2$ 非常小，经过矩阵 $W$ 变换后，输出的扰动 $|W \Delta x|_2$ 也会被放大很多倍。这意味着原始输入中微小的误差或噪声，在经过这个矩阵层之后，可能会被显著放大，导致输出结果的剧烈变化。

总结来说：

大 $|W|_2$ 意味着强放大效应：矩阵 $W$ 有能力将某些方向上的输入向量放大很多倍。
误差传播：当输入存在小扰动时（比如量化误差、浮点数误差或传感器噪声），这个扰动会被 $|W|_2$ 的大小所放大，从而导致最终输出的误差也很大。
在推理系统中，尤其是在量化或低精度计算时，如果模型的某个权重矩阵的谱范数过大，那么该层将对量化噪声、数值舍入误差非常敏感，容易导致精度大幅下降或输出不稳定。因此，在模型设计或优化过程中，有时会通过正则化等方式来限制权重的谱范数，以提高模型的鲁棒性和数值稳定性。
- 在量化/剪枝时，这种层更容易出现数值不稳定 / 输出来回抖动  
- 有些鲁棒性分析会用谱范数做 Lipschitz 常数的上界

**解释**：

*什么是 Lipschitz 常数*： 在数学中，一个函数 $f: X \to Y$ 是 Lipschitz 连续的，如果存在一个常数 $K \ge 0$，使得对于 $X$ 域中的任意两个点 $x_1$ 和 $x_2$，都有：

$$\|f(x_1) - f(x_2)\| \le K \|x_1 - x_2\|$$
这个常数 $K$ 就被称为该函数的 Lipschitz 常数。这里的 $\|\cdot\|$ 表示某种范数（例如向量的欧几里得范数或矩阵的谱范数）。

*直观理解*： Lipschitz 常数 $K$ 给出了函数 “最陡峭” 的程度。它限制了函数值变化的速率：

如果 $K$很小，函数是“平缓”的，输出的变化不会比输入的微小变化大太多。它像一个斜率有限的函数。
如果 $K$很大，函数可能是“陡峭”的，输出的变化可能比输入的微小变化大很多。这意味着函数对输入的小扰动非常敏感。

*数值稳定性与误差传播*：如果一个矩阵（可以看作一个线性函数）的谱范数很大，那么它会将输入的微小误差放大很多倍。一个线性函数 $f(x) = Wx$ 的 Lipschitz 常数就是它的谱范数 $|W|_2$。
对于非线性函数（如激活函数、整个神经网络），其 Lipschitz 常数限制了输入噪声或量化误差在网络中传播时的最大放大倍数。
低 Lipschitz 常数 的模型通常具有更好的 数值稳定性和对抗样本的鲁棒性。

*模型压缩与量化*：在量化或剪枝过程中，我们会对模型参数引入微小的扰动（量化误差）。如果某一层或整个模型的 Lipschitz 常数过大，这些微小扰动就可能被显著放大，导致模型精度急剧下降。
因此，在进行模型压缩时，有时会关注保持或限制模型的 Lipschitz 常数，以确保压缩后的模型仍然稳定且性能良好。在实践中，直接计算复杂深度学习模型的精确 Lipschitz 常数通常非常困难（NP-hard）。
但可以通过一些方法来 估计其上界 或 正则化 来间接限制它，例如：谱范数正则化 (Spectral Normalization)：限制神经网络中权重矩阵的谱范数，从而限制线性层的 Lipschitz 常数。 某些激活函数（如 ReLU）是 1-Lipschitz 的，有助于控制传播的放大。Batch Normalization 或 Layer Normalization 也有助于稳定激活的分布，间接控制函数的行为。总而言之，Lipschitz 常数是衡量函数平滑度和对输入扰动敏感程度的一个重要指标。


---

### 1.4.3 $L_1$ 和 $L_0$ 范数（稀疏性）

- 元素级 $L_1$：

$$
\|W\|_1 = \sum_{i,j} |w_{ij}|
$$


- “$L_0$”（非零个数）：

$$
\|W\|_0 = \#\{(i,j) : w_{ij} \ne 0\}
$$


**【工程视角】**  

- $L_0$ 直接度量“有多少元素被保留/剪掉”，但对应的优化问题是 NP-hard
- 通常用 $L_1$ 作为凸近似：

$$
L(W) + \lambda \|W\|_1
$$


  训练后很多元素自然趋近于 0，方便做后处理剪枝。

**【具体解释】**
1. $L_0$ 范数：理想但难优化
定义：$\|W\|_0$ 严格来说不是一个范数，它表示矩阵 $W$ 中 非零元素的个数。
目标：在剪枝中，我们希望最小化非零元素的数量，从而让模型变得稀疏，这样可以减少存储和计算。所以优化目标可能是这样的：
$$
\min_{\widehat{W}} \; L(\widehat{W}) \;+\; \lambda \,\|\widehat{W}\|_{0}
$$

问题：$L_0$ 范数是非凸、不连续的，这意味着它没有良好的数学性质，导致优化问题是 NP-hard 的（即计算上非常困难，没有高效的全局最优解算法）。你无法直接使用梯度下降等基于梯度的优化方法来最小化它，因为它的梯度几乎处处为零或不存在。

2. $L_1$ 范数：凸且促进稀疏性
定义：$\|W\|1$ 表示矩阵 $W$ 中所有元素的 绝对值之和：
$$
\|W\|_1 = \sum_{i,j} |w_{ij}|
$$

性质：

凸性：$L_1$ 范数是一个 凸函数。这意味着当我们将它作为正则项加入损失函数时，整个优化问题（如果是凸损失函数）将仍然是凸的，或者至少是更容易优化的。我们可以使用梯度下降及其变种（例如次梯度下降，因为 $|x|$ 在 $x=0$ 处不可导）来找到全局最优解或近似最优解。

促进稀疏性：虽然 $L_1$ 范数不直接统计非零元素的个数，但它有一个非常重要的性质：在优化过程中，为了最小化 $\sum |w_{ij}|$，优化器倾向于将许多权重推向 精确的零，而不是一个非常小但非零的值。这是因为 $|x|$ 在 $x=0$ 附近有一个“尖点”，这使得优化器在接近零时受到更大的惩罚，从而促使权重跨过零点。

3. 为什么是“凸近似”？
近似 $L_0$ 行为：虽然 $L_1$ 不等于 $L_0$，但它能有效地诱导稀疏性，从而在实践中近似达到我们用 $L_0$ 追求的效果（即得到一个有很多零的模型）。
可优化性： $L_1$ 是凸的，这使得优化问题变得可解。它提供了一个数学上更容易处理的替代方案，可以在合理的时间内找到一个好的解。所以，当看到剪枝的优化目标中包含 $L_1$ 正则项时，比如：
$$
\min_{\widehat{W}} \; L(\widehat{W}) + \lambda \|\widehat{W}\|_1
$$
它的意思是：我们希望在保持模型精度的同时（最小化 $L(\widehat{W})$），也能让模型的权重尽可能小，并且倾向于零（最小化 $\lambda |\widehat{W}|_1$），从而达到稀疏化的目的。这种方法是实践中实现模型稀疏化（尤其是非结构化剪枝）非常常见且有效的方式。

---

### 1.4.4 在工程中如何实际使用这些范数？

1. **判断某层是否适合低秩分解**  
   - 计算奇异值 $\{\sigma_i\}$，看前 k 个是否已经占到整体能量的绝大部分：

$$
\frac{\sum_{i=1}^k \sigma_i^2}{\sum_{i=1}^r \sigma_i^2} \approx 0.95 \text{ 或更高}
$$


2. **设计剪枝策略**  
   - 若某层权重的 $L_1$ 分布极不均匀（大量接近 0 的元素），说明存在自然稀疏性 → 可以安全剪枝
   - 可以对每一行/每一列计算 $L_1$ 或 $L_2$ norm，作为“通道重要性”的指标

3. **量化敏感性分析**  
   - 对于谱范数较大的层，可以考虑：
     - 使用更高 bit 的量化（例如 8-bit 而不是 4-bit）
     - 或做更细致的 per-channel scaling 来减小量化误差

> 总结：范数帮助你从“拍脑袋选层/选 rank/选 bit”  
> 变成“有指标、有依据的工程决策”。


### 1.4.5 例子
三类范数的工程决策整合例子。
假设有一个 Transformer FFN 层 𝑊
计算结果如下

| 指标           | 数值           | 解释                |
| ------------ | ------------ | ----------------- |
| 奇异值能量 E(128) | 96.3%        | 很适合低秩分解（rank=128） |
| 平均 L1 norm   | 大量行在 0.01 以下 | 有稀疏性，可做剪枝         |
| 最大奇异值（谱范数）   | 780          | 量化敏感，不要用太低 bit    |

最终工程决策可能是
🔹 第一步：低秩分解

把 4096×4096 降到 rank=128
→ 参数从 16M → 1M
→ FLOPs 从 33M → 2M

🔹 第二步：结构化剪枝

由于 L1 分布稀疏 → 去掉 20% 无用通道
→ 通道数从 4096 → 3276

🔹 第三步：量化

因为谱范数大 → 不能用 INT4
✔ 选 FP16 或 INT8-per-channel

最终效果：

模型大小减少约 16× × 1.25× = 20×

FLOPs 降到原来的 1/10~1/15

延迟显著降低，没有明显精度损失


In [1]:
import torch
import torch.nn as nn
import numpy as np

def get_weight_2d(layer: nn.Module):
    """
    flatten the weight of a layer into 2D matrix
    """
    W = layer.weight.detach().float().clone()

    if isinstance(layer, nn.Linear):
        # [out_features, in_features]
        W2d = W  # already 2D
    elif isinstance(layer, nn.Conv2d):
        # [out_channels, in_channels, kh, kw] -> [out_channels, in_channels*kh*kw]
        out_c, in_c, kh, kw = W.shape
        W2d = W.view(out_c, in_c * kh * kw)
    else:
        raise NotImplementedError(f"Unsupported layer type: {type(layer)}")

    return W2d


def analyze_layer(layer: nn.Module,
                  energy_target: float = 0.98,
                  max_rank_ratio: float = 0.5,
                  max_prune_ratio: float = 0.5):
    """
    for a given layer,
    - SVD analysis → recommend low-rank
    - L1 distribution analysis → recommend pruning ratio (by output channels)
    - Spectral norm analysis → recommend quantization bits
    """
    W2d = get_weight_2d(layer)                  # [out_dim, in_dim]
    out_dim, in_dim = W2d.shape

    # ===== 1. SVD & rank recommendation =====
    # only need singular values, and skip U and V
    # note: for very large matrices, consider using randomized
    sigma = torch.linalg.svdvals(W2d)           # [min(out_dim, in_dim)]
    sigma2 = sigma ** 2
    total_energy = sigma2.sum()
    cum_energy = torch.cumsum(sigma2, dim=0) / total_energy

    # Find the smallest k such that cum_energy[k-1] >= energy_target
    k_candidate = int((cum_energy >= energy_target).nonzero(as_tuple=True)[0][0].item() + 1)
    
    # Do not exceed max_rank_ratio * min_dim to prevent keeping too many components
    max_rank = int(min(out_dim, in_dim) * max_rank_ratio)
    k_recommended = min(k_candidate, max_rank)

    # ===== 2. L1 distribution & pruning ratio recommendation (by output channels) =====
    # compute L1 norm for each output channel
    l1_per_out = W2d.abs().sum(dim=1)          # [out_dim]
    l1_np = l1_per_out.cpu().numpy()
    # compute several percentiles to sense sparsity
    q10, q25, q50 = np.percentile(l1_np, [10, 25, 50])

    # If the gap between the low percentile and the median is large,
    # it indicates that the tail channels are not important and can be pruned.
    # Simple heuristic: check if the 25% percentile is significantly smaller than the 50% percentile.
    prune_ratio = 0.0
    if q25 < 0.3 * q50:
        # very sparse, can prune up to 30%
        prune_ratio = min(0.3, max_prune_ratio)
    elif q25 < 0.5 * q50:
        # slightly sparse, can prune up to 15%
        prune_ratio = min(0.15, max_prune_ratio)
    else:
        prune_ratio = 0.0  # 不建议剪

    # ===== Spectral norm & quantization bits recommendation =====
    spectral_norm = sigma.max().item()

    # Very rough heuristic:
    if spectral_norm < 5:
        quant_bits = 4 
    elif spectral_norm < 20:
        quant_bits = 8 
    else:
        # Recommended minimally 8 bits (or even 16 bits) with per-channel scaling
        quant_bits = 8  

    # print results
    print("=" * 60)
    print(f"Layer: {layer.__class__.__name__}, weight shape = {tuple(layer.weight.shape)}")
    print("--- Low-rank analysis ---")
    print(f"  min(out_dim, in_dim)  = {min(out_dim, in_dim)}")
    print(f"  target energy         = {energy_target:.3f}")
    print(f"  k (reach target)      = {k_candidate}")
    print(f"  k (capped by {max_rank_ratio:.2f}*min_dim) = {k_recommended}")
    print(f"  Compression ratio     ≈ {min(out_dim, in_dim) / k_recommended:.2f}x (in FLOPs/params)")

    print("\n--- L1 sparsity / pruning analysis ---")
    print(f"  L1 per-out quantiles  (10%,25%,50%) = ({q10:.4g}, {q25:.4g}, {q50:.4g})")
    print(f"  Recommended prune ratio (by channels) ≈ {prune_ratio*100:.1f}%")

    print("\n--- Quantization sensitivity ---")
    print(f"  Spectral norm ||W||_2 = {spectral_norm:.4g}")
    print(f"  Recommended quant bits = {quant_bits}-bit")

    return {
        "sigma": sigma,
        "cum_energy": cum_energy,
        "recommended_rank": k_recommended,
        "recommended_prune_ratio": prune_ratio,
        "spectral_norm": spectral_norm,
        "recommended_quant_bits": quant_bits,
    }


try:
    from torchvision.models import resnet18
    # load no pre-trained weights
    model = resnet18(weights=None)  
except Exception as e:
    raise e

torch.manual_seed(0)

# Traverse all sub-modules
for name, module in model.named_modules():
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        print(f"\n>>> Analyzing layer: {name}")
        analyze_layer(module, energy_target=0.98, max_rank_ratio=0.5, max_prune_ratio=0.5)



>>> Analyzing layer: conv1
Layer: Conv2d, weight shape = (64, 3, 7, 7)
--- Low-rank analysis ---
  min(out_dim, in_dim)  = 64
  target energy         = 0.980
  k (reach target)      = 58
  k (capped by 0.50*min_dim) = 32
  Compression ratio     ≈ 2.00x (in FLOPs/params)

--- L1 sparsity / pruning analysis ---
  L1 per-out quantiles  (10%,25%,50%) = (2.676, 2.822, 2.921)
  Recommended prune ratio (by channels) ≈ 0.0%

--- Quantization sensitivity ---
  Spectral norm ||W||_2 = 0.4857
  Recommended quant bits = 4-bit

>>> Analyzing layer: layer1.0.conv1
Layer: Conv2d, weight shape = (64, 64, 3, 3)
--- Low-rank analysis ---
  min(out_dim, in_dim)  = 64
  target energy         = 0.980
  k (reach target)      = 62
  k (capped by 0.50*min_dim) = 32
  Compression ratio     ≈ 2.00x (in FLOPs/params)

--- L1 sparsity / pruning analysis ---
  L1 per-out quantiles  (10%,25%,50%) = (25.84, 26.56, 26.97)
  Recommended prune ratio (by channels) ≈ 0.0%

--- Quantization sensitivity ---
  Spectral nor

## 1.5 卷积的线性代数表达：Conv = MatMul

卷积层（Conv2d）是 CNN 和很多视觉模型的核心。  
从数学上，它是一个带有结构的线性算子：

$$
y = K * x
$$


- $x$：输入特征图（如 $\mathbb{R}^{C_{\text{in}} \times H \times W}$）  
- $K$：卷积核（如 $\mathbb{R}^{C_{\text{out}} \times C_{\text{in}} \times k_h \times k_w}$）

通过适当展开和重排，可以把它改写为**矩阵乘法**。

---

### 1.5.1 im2col：把局部 patch 展成列向量

以单通道、kernel=2×2、stride=1 为例：

输入：

```text
1 2 3
4 5 6
7 8 9
```

卷积核：

```text
k11 k12
k21 k22
```

所有 2×2 patch：

1. 上左：$\{1,2,4,5\}$
2. 上右：$\{2,3,5,6\}$
3. 下左：$\{4,5,7,8\}$
4. 下右：$\{5,6,8,9\}$

我们把每个 patch 按固定顺序展开成列向量（也就是从左上开始，窗口大小 2×2，水平/垂直各滑动一步，枚举所有可能的 2×2 patch）：

$$
x_1 = [1,2,4,5]^\top, \quad
x_2 = [2,3,5,6]^\top, \quad
x_3 = [4,5,7,8]^\top, \quad
x_4 = [5,6,8,9]^\top
$$


然后组成一个矩阵 $X_{\text{col}} = \{x_1, x_2, x_3, x_4\}$：

$$
X_{\text{col}} =
\begin{bmatrix}
1 & 2 & 4 & 5 \\
2 & 3 & 5 & 6 \\
4 & 5 & 7 & 8 \\
5 & 6 & 8 & 9
\end{bmatrix}
\in \mathbb{R}^{4 \times 4}
$$


卷积核也展开成向量：

$$
w = [k_{11}, k_{12}, k_{21}, k_{22}]
$$


则每个输出位置：

$$
y_i = w x_i
$$


所有位置一起写成矩阵形式：

$$
y^\top = w X_{\text{col}}
$$


---

### 1.5.2 多通道 / 多卷积核的情况

对于一般情况：

- 输入：$x \in \mathbb{R}^{C_{\text{in}} \times H \times W}$
- 卷积核：$K \in \mathbb{R}^{C_{\text{out}} \times C_{\text{in}} \times k_h \times k_w}$

通过 im2col：

- 生成矩阵 $X_{\text{col}} \in \mathbb{R}^{(C_{\text{in}} k_h k_w) \times L}$  
  其中 L 是滑窗数量（输出空间尺寸的乘积）
- 卷积核压平成矩阵 $W \in \mathbb{R}^{C_{\text{out}} \times (C_{\text{in}} k_h k_w)}$

则卷积可以写成：

$$
Y = W X_{\text{col}}
$$


- $Y \in \mathbb{R}^{C_{\text{out}} \times L}$，之后 reshape 回输出尺寸

**【核心结论】**  
> Conv2d 经过 im2col 变换后，本质就是一个 GEMM。

---
### 1.5.3 多核多通道例子
设：
- 输入特征图：$x \in \mathbb{R}^{C_{\text{in}} \times H \times W},\quad
C_{\text{in}} = 2,\ H=W=3$
- 卷积核： $K \in \mathbb{R}^{C_{\text{out}} \times C_{\text{in}} \times k_h \times k_w},\quad
C_{\text{out}} = 2,\ k_h=k_w=2$
- stride = 1, padding = 0
- 输出空间尺寸：$H_{\text{out}} = W_{\text{out}} = 3-2+1 = 2$
所以一共 $L = H_{\text{out}} \cdot W_{\text{out}} = 4$ 个输出位置。
---
1. 输入数据（2 个通道）
- 第 1 个通道（c=0）：
$\begin{array}{c} x^{(0)} =
\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix} \end{array}$

- 第 2 个通道（c=1）：
$\begin{array}{c} x^{(1)} =
\begin{bmatrix}
10 & 11 & 12 \\
13 & 14 & 15 \\
16 & 17 & 18
\end{bmatrix} \end{array}$
---
2. 卷积核（2 个输出通道 = 2 个 filter）
每个卷积核对两个输入通道都有自己的 2×2 kernel：
- 第 1 个输出通道（filter 0）：
$\begin{array}{c} K^{(0,0)} =
\begin{bmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}
\end{bmatrix},
\quad
K^{(0,1)} =
\begin{bmatrix}
b_{11} & b_{12} \\
b_{21} & b_{22}
\end{bmatrix} \end{array}$

- 第 2 个输出通道（filter 1）：
$\begin{array}{c} K^{(1,0)} =
\begin{bmatrix}
c_{11} & c_{12} \\
c_{21} & c_{22}
\end{bmatrix},
\quad
K^{(1,1)} =
\begin{bmatrix}
d_{11} & d_{12} \\
d_{21} & d_{22}
\end{bmatrix} \end{array}$

这里 $(0,0)$  表示：输出通道 0，对输入通道 0 的 kernel， $(0,1)$ 表示：输出通道 0，对输入通道 1 的 kernel，依此类推。

---

3. 用 im2col 展开多通道 patch → 列向量
输出空间是 2×2，所以有 4 个滑窗位置：

- 左上：覆盖输入坐标 (0:2, 0:2)

- 右上：覆盖 (0:2, 1:3)

- 左下：覆盖 (1:3, 0:2)

- 右下：覆盖 (1:3, 1:3)

对每个位置，我们要从 两个通道 各取一个 2×2 patch，然后全部摊平接在一起，形成一个长度为 $C_{\text{in}} \cdot k_h \cdot k_w = 2 \cdot 2 \cdot 2 = 8$ 的列向量。

🔹 位置 1：左上 patch（top-left）
- 通道 0 的 2×2 patch：
$\begin{bmatrix}
1 & 2 \\
4 & 5
\end{bmatrix}$
- 通道 1 的 2×2 patch：
$\begin{bmatrix}
10 & 11 \\
13 & 14
\end{bmatrix}$

按固定顺序（比如：通道顺序优先，每个 patch 按行展平）摊平并拼接：
$\begin{array}{c} x_{\text{col},1} =
\begin{bmatrix}
1 \\ 2 \\ 4 \\ 5 \\ 10 \\ 11 \\ 13 \\ 14
\end{bmatrix}
\in \mathbb{R}^{8} \end{array}$

🔹 位置 2：右上 patch（top-right）
- 通道 0：$\begin{bmatrix} 2 & 3 \\ 5 & 6 \end{bmatrix}$
- 通道 1：$\begin{bmatrix} 11 & 12 \\ 14 & 15 \end{bmatrix}$
展开：
$\begin{array}{c} x_{\text{col},2} =
\begin{bmatrix}
2 \\ 3 \\ 5 \\ 6 \\ 11 \\ 12 \\ 14 \\ 15
\end{bmatrix} \end{array}$

🔹 位置 3：左下 patch（bottom-left）
- 通道 0：$\begin{bmatrix} 4 & 5 \\ 7 & 8 \end{bmatrix}$
- 通道 1：$\begin{bmatrix} 13 & 14 \\ 16 & 17 \end{bmatrix}$
展开：
$\begin{array}{c} x_{\text{col},3} =
\begin{bmatrix}
4 \\ 5 \\ 7 \\ 8 \\ 13 \\ 14 \\ 16 \\ 17
\end{bmatrix} \end{array}$

🔹位置 4：右下 patch（bottom-right）
- 通道 0：$\begin{bmatrix} 5 & 6 \\ 8 & 9 \end{bmatrix}$
- 通道 1：$\begin{bmatrix} 14 & 15 \\ 17 & 18 \end{bmatrix}$
展开：
$\begin{array}{c} x_{\text{col},4} =
\begin{bmatrix}
5 \\ 6 \\ 8 \\ 9 \\ 14 \\ 15 \\ 17 \\ 18
\end{bmatrix} \end{array}$

🔹 组合成 $X_{\text{col}}$ 把 4 个列向量拼在一起：
$\begin{array}{c} X_{\text{col}} =
\begin{bmatrix}
1  & 2  & 4  & 5  \\
2  & 3  & 5  & 6  \\
4  & 5  & 7  & 8  \\
5  & 6  & 8  & 9  \\
10 & 11 & 13 & 14 \\
11 & 12 & 14 & 15 \\
13 & 14 & 16 & 17 \\
14 & 15 & 17 & 18
\end{bmatrix}
\in \mathbb{R}^{8 \times 4} \end{array}$

---
4。 卷积核展平成矩阵 W（多卷积核 → 多行）
每个输出通道的 filter 是 两个 2×2 的 kernel（对应 2 个输入通道），我们也把它们展平并拼接：
- 对输出通道 0（filter 0）：
$\begin{array}{c} w^{(0)} =
\begin{bmatrix}
a_{11} \\ a_{12} \\ a_{21} \\ a_{22} \\
b_{11} \\ b_{12} \\ b_{21} \\ b_{22}
\end{bmatrix}^\top \end{array}$	​
- 对输出通道 1（filter 1）：
$\begin{array}{c} w^{(1)} =
\begin{bmatrix}
c_{11} \\ c_{12} \\ c_{21} \\ c_{22} \\
d_{11} \\ d_{12} \\ d_{21} \\ d_{22}
\end{bmatrix}^\top \end{array}$

- 这样组合成矩阵$W$：
$\begin{array}{c} W =
\begin{bmatrix}
w^{(0)} \\
w^{(1)}
\end{bmatrix}
\in \mathbb{R}^{2 \times 8} \end{array}$
---

5. 统一成一个矩阵乘法：Y = W X_col
现在卷积运算可以写成：$Y = W X_{\text{col}}$

- $W \in \mathbb{R}^{C_{\text{out}} \times (C_{\text{in}} k_h k_w)} = \mathbb{R}^{2 \times 8}$
- $X_{\text{col}} \in \mathbb{R}^{(C_{\text{in}} k_h k_w) \times L} = \mathbb{R}^{8 \times 4}$
- $Y \in \mathbb{R}^{C_{\text{out}} \times L} = \mathbb{R}^{2 \times 4}$
其中：
- 第 1 行对应输出通道 0 在所有 4 个空间位置的输出
- 第 2 行对应输出通道 1
- 再把每一行 reshape 回 2×2，就得到标准的 2×2 feature map：$Y^{(c)} \in \mathbb{R}^{2 \times 2},\quad c=0,1$

### 1.5.4 在推理加速中的意义

1. **重用高性能 GEMM 内核**  
   - 不需要为每种卷积形状写全新的 kernel  
   - 只需实现 im2col（或更高级的变换）+ 调用 GEMM
2. **统一优化目标**  
   - GEMM 的 tiling、向量化、cache 利用一旦写好，Conv 也能享受
3. **更容易做量化/剪枝**  
   - 权重在 im2col 形式下就是一个矩阵 W，方便 per-channel / per-row 量化和剪枝

> 在高性能推理库中，Conv2d 的实现通常要么：  
> - 走 im2col + GEMM 路线，  
> - 要么走专门的 convolution kernel，但其内部仍然围绕“块矩阵乘法”的思想设计。

---




In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

# ======================
# 1. 定义一个简单的 Conv2d
# ======================
in_channels = 2
out_channels = 3
kernel_size = 3
stride = 1
padding = 1

conv = nn.Conv2d(
    in_channels=in_channels,
    out_channels=out_channels,
    kernel_size=kernel_size,
    stride=stride,
    padding=padding,
    bias=True,
)

# 构造输入：batch_size=1, C=2, H=W=4
x = torch.randn(1, in_channels, 4, 4)

# 用 nn.Conv2d 直接算一遍（标准实现）
y_conv = conv(x)  # 形状 [1, out_channels, H_out, W_out]
print("y_conv shape:", y_conv.shape)

# ======================================
# 2. 用 im2col + matmul 手工实现同样的卷积
# ======================================

# unfold 相当于 im2col：
# x_unfold: [N, C_in * k_h * k_w, L]
# 其中 L = H_out * W_out
x_unfold = F.unfold(
    x,
    kernel_size=kernel_size,
    dilation=1,
    padding=padding,
    stride=stride,
)
# 取一下展开后的尺寸
N, K, L = x_unfold.shape  # K = C_in * k_h * k_w
print("x_unfold shape:", x_unfold.shape)  # [N, K, L]

# 卷积核权重： [C_out, C_in, k_h, k_w] -> [C_out, K]
W = conv.weight.view(out_channels, -1)  # [C_out, C_in * k_h * k_w]
b = conv.bias                           # [C_out]

# 对每个 batch 独立做矩阵乘法：
# 对某个样本 n：
#   x_unfold[n]: [K, L]
#   输出 feature: [C_out, L] = W @ x_unfold[n] + b
y_im2col_list = []
for n in range(N):
    # [C_out, L] = [C_out, K] @ [K, L]
    y_n = W @ x_unfold[n]  # [C_out, L]
    # 加 bias：在第 0 维 broadcast
    y_n = y_n + b.view(-1, 1)  # [C_out, L]
    y_im2col_list.append(y_n)

# 堆回 batch 维： [N, C_out, L]
y_im2col = torch.stack(y_im2col_list, dim=0)

# 将 L 展回 H_out, W_out
H_out = y_conv.shape[2]
W_out = y_conv.shape[3]
y_im2col = y_im2col.view(N, out_channels, H_out, W_out)

print("y_im2col shape:", y_im2col.shape)

# ============================
# 3. 对比 nn.Conv2d 与 im2col+matmul
# ============================

diff = (y_conv - y_im2col).abs().max().item()
print("max |y_conv - y_im2col| =", diff)

if diff < 1e-5:
    print("✅ Conv2d == im2col + matmul (within numerical tolerance)")
else:
    print("⚠️ there is a discrepancy between Conv2d and im2col + matmul!")


y_conv shape: torch.Size([1, 3, 4, 4])
x_unfold shape: torch.Size([1, 18, 16])
y_im2col shape: torch.Size([1, 3, 4, 4])
max |y_conv - y_im2col| = 0.0
✅ Conv2d == im2col + matmul (within numerical tolerance)


## 1.6 Attention 的线性代数结构

以单头 self-attention 为例（不考虑 bias & mask）：

$$
Q = X W_Q,\quad
K = X W_K,\quad
V = X W_V \\
A = \text{softmax}\left(\frac{Q K^\top}{\sqrt{d_k}}\right) \\
O = A V
$$


- $X \in \mathbb{R}^{T \times d_{\text{model}}}$：输入序列
- $W_Q, W_K, W_V \in \mathbb{R}^{d_{\text{model}} \times d_k}$
- $Q,K,V \in \mathbb{R}^{T \times d_k}$
- $A \in \mathbb{R}^{T \times T}$
- $O \in \mathbb{R}^{T \times d_v}$

其中：

1. $Q = X W_Q$ 是一个 GEMM（$T \times d_{\text{model}}$ 乘 $d_{\text{model}} \times d_k$）
2. $K = X W_K$、$V = X W_V$ 同理
3. $QK^\top$ 是一个 **非常大的 GEMM**（$T \times d_k$ 乘 $d_k \times T$）
4. $A V$ 又是一个 GEMM（$T \times T$ 乘 $T \times d_v$）
- ${d_{\text{model}}}$: 模型的隐藏维度，每个 token 的向量长度（Transformer 主通道维度）。
- ${d_k}$: 单个注意力头的 Query/Key 投影维度（通常 ${d_{\text{model}}}$ / num_heads）。
- ${d_v}$: Value 投影维度（常设成和 ${d_k}$ 相等）。
- ${T}$: 序列长度（时间步数/ token 数


**【工程视角】**  
- Attention 中真正“贵”的地方就是两个矩阵乘法：$QK^\top$ 和 $A V$
- softmax 本身 FLOPs 不多，但存在数值稳定性的挑战（第 8 章展开）
- 所有对 Attention 的加速（如 FlashAttention、各种 kernel fusion）本质都是：
  - 减少显存读写
  - 改善访问模式
  - 在不显式构造 $T \times T$ 矩阵的前提下，实现等价的线性代数运算

> 一旦站在“线性代数”视角看 Attention，就更容易理解各种加速论文的思路。

---

## 1.7 Embedding：查表 = 选行 = 稀疏 GEMM

Embedding 矩阵：$E \in \mathbb{R}^{V \times D}$

- V：词表大小
- D：隐层维度 / embedding 维度

给定 token id = i，Embedding 的作用就是：

$$
\text{Embedding}(i) = E_i
$$


即：**从矩阵 E 中选取第 i 行**。

如果构造一个 one-hot 向量 $x \in \mathbb{R}^V$：

$$
x_j = \begin{cases}
1, & j = i \\
0, & \text{else}
\end{cases}
$$


则：

$$
y = x^\top E = E_i
$$


也就是说：

> Embedding 在数学上等价于：**one-hot 向量乘 embedding 矩阵的矩阵乘法（一个极其稀疏的 GEMM）**。

**【工程视角】**  

- 对 Embedding 来说，算力（FMA）几乎不构成瓶颈，因为没有大规模乘加
- 真正的瓶颈是：
  - 内存访问（随机访问 E 的不同行）
  - cache 命中率
  - 带宽

这解释了为什么：  
> 巨大词表的 LLM，在部署时经常是 “embedding 和 LM head 很吃带宽”；  
> 而中间层（GEMM）则更吃计算。

---

## 1.8 数据布局（Memory Layout）与线性代数

矩阵在内存中可以有两种常见布局：

- row-major（行优先，C-style）
- column-major（列优先，Fortran/BLAS-style）

附：
BLAS（Basic Linear Algebra Subprograms）是密集线性代数的标准 API 规范，一组基础算子库接口：

- Level 1/2/3：向量运算、矩阵‑向量、矩阵‑矩阵（GEMM 在 Level 3）。
- 不同平台有不同实现：OpenBLAS/BLIS、MKL、Apple Accelerate、cuBLAS 等，提供高度优化的内核。

对于 $A \in \mathbb{R}^{M \times K}$：

- row-major：同一行的元素连续存放
- column-major：同一列的元素连续存放

这会影响：

- 连续访问的方向（stride=1）
- 缓存局部性（cache locality）
- 是否容易被 SIMD 向量化

**【工程视角】**  

- 很多 GEMM 内核会要求 A/B/C 矩阵采用特定布局（如 A row-major，B col-major），以便：
  - 在 inner-k loop 中连续访问数据
  - 利用硬件的向量加载指令（如 `ld1q`、`vmovaps` 等）
- 这就是为什么你在 kernel 代码中会看到大量 transpose / layout transform，它们本质是为线性代数运算创建“更友好的 memory layout”。

---

## 1.9 小结：线性代数是 Runtime Inference 的共同语言

本章核心结论：

1. **推理的大部分 FLOPs 都来自 GEMM（矩阵乘法）**  
   - 全连接 / MLP / Attention projection / Conv2d (im2col) 等
2. **低秩近似（SVD）是最重要的压缩数学之一**  
   - 通过 rank-k 近似，把大矩阵拆成两个小矩阵乘法
3. **矩阵范数给出度量压缩误差的工具**  
   - Frobenius、谱范数、L1/L0
4. **Conv、Attention、Embedding 都可以在“矩阵视角”下统一看待**
5. **数据布局与线性代数紧密相关**  
   - Row-major / col-major / tiling 决定是否能高效利用硬件

> 接下来几章会在这个基础上，继续引入：  
> - 为何这些矩阵要被“优化”（数值优化章节）  
> - 为何我们敢于用粗糙的近似（近似理论章节）  
> - 如何在概率/信息论框架下理解压缩与量化  
> - 如何结合硬件特性真正做到“跑满”推理芯片